# DeepSeek-R1 파인튜닝 모델 추론 (Inference)

이 노트북은 파인튜닝된 DeepSeek-R1 모델을 사용하여 페르소나 기반 응답을 생성하는 코드를 제공합니다.

## 필요한 라이브러리 설치

먼저 필요한 라이브러리들을 설치합니다:

In [ ]:
!pip install torch transformers peft

## 라이브러리 임포트

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

## 모델 로드 함수

In [ ]:
def load_model(model_path):
    """
    학습된 모델과 토크나이저를 로드합니다.
    """
    print(f"Loading model from {model_path}...")
    
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    
    # LoRA 어댑터가 저장된 모델인 경우
    try:
        # PeftModel 설정 로드
        config = PeftConfig.from_pretrained(model_path)
        # 기본 모델 로드
        base_model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path, 
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
        # LoRA 어댑터 적용
        model = PeftModel.from_pretrained(base_model, model_path)
    except:
        # 전체 모델이 저장된 경우
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16,
            device_map="auto",
            trust_remote_code=True
        )
    
    return model, tokenizer

## 응답 생성 함수

In [ ]:
def generate_response(model, tokenizer, persona, question, 
                     max_length=512, temperature=0.7, top_p=0.95):
    """
    입력된 페르소나와 질문에 대한 응답을 생성합니다.
    """
    # 입력 형식 구성
    input_text = f"<persona>\n{persona}\n</persona>\n\n<human>\n{question}\n</human>\n\n<assistant>\n"
    
    # 토큰화
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    # 응답 생성
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=top_p,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
        )
    
    # 응답 디코딩
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 응답 부분만 추출
    assistant_part = full_response.split("<assistant>")[-1].strip()
    
    return assistant_part

## 모델 로드

파인튜닝된 모델을 로드합니다. 모델 경로를 필요에 따라 수정하세요.

In [ ]:
model_path = "./deepseek-r1-finetuned"

try:
    model, tokenizer = load_model(model_path)
    print("모델 로드 성공!")
except Exception as e:
    print(f"모델 로드 중 오류 발생: {e}")
    print("대신 원본 모델을 로드합니다.")
    model, tokenizer = load_model("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

## 페르소나 설정

모델이 응답할 때 사용할 페르소나를 정의합니다.

In [ ]:
# 기본 페르소나 설정
default_persona = "20대 남성, 대학생, 취미는 게임과 독서, 성격은 친절하고 차분함"

# 사용자가 원하는 페르소나 입력 가능
custom_persona = input("페르소나를 입력하세요 (비워두면 기본값 사용): ").strip()

persona = custom_persona if custom_persona else default_persona
print(f"사용할 페르소나: {persona}")

## 단일 대화 테스트

페르소나와 질문을 통해 응답을 생성해봅니다.

In [ ]:
question = "안녕하세요, 자기소개 좀 해주세요."

response = generate_response(model, tokenizer, persona, question, 
                            temperature=0.7, top_p=0.95)

print(f"질문: {question}")
print(f"응답: {response}")

## 대화형 채팅 인터페이스

사용자와 대화형으로 채팅할 수 있는 인터페이스를 제공합니다.

In [ ]:
print("\n====== DeepSeek-R1 페르소나 채팅 ======\n")
print("대화를 시작합니다. 종료하려면 'quit' 또는 'exit'를 입력하세요.\n")

while True:
    # 질문 입력
    question = input("사용자: ").strip()
    if question.lower() in ["quit", "exit"]:
        print("채팅을 종료합니다.")
        break
    
    # 응답 생성
    try:
        response = generate_response(model, tokenizer, persona, question)
        print(f"어시스턴트: {response}\n")
    except Exception as e:
        print(f"응답 생성 중 오류 발생: {e}")

## 생성 파라미터 조정 실험

다양한 파라미터로 응답을 생성하여 결과를 비교해볼 수 있습니다.

In [ ]:
test_question = "오늘 기분이 어때요?"

# 온도(temperature) 값 변경 실험
temperatures = [0.3, 0.7, 1.0]

print("온도(temperature) 변경에 따른 응답 비교:")
for temp in temperatures:
    response = generate_response(model, tokenizer, persona, test_question, 
                              temperature=temp, top_p=0.95)
    print(f"\n온도 {temp}일 때 응답:\n{response}")

In [ ]:
# top_p 값 변경 실험
top_p_values = [0.7, 0.9, 0.99]

print("top_p 변경에 따른 응답 비교:")
for top_p_val in top_p_values:
    response = generate_response(model, tokenizer, persona, test_question, 
                              temperature=0.7, top_p=top_p_val)
    print(f"\ntop_p {top_p_val}일 때 응답:\n{response}")